In [1]:
import json
from authorization import SpotifyAuth

In [2]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

In [3]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

# Get User Details

In [4]:
import spotify_functions as spotify

In [5]:
spotify.get_user_details(spotify_authorisation.get_access_token())

{'display_name': 'sdspot2034',
 'external_urls': {'spotify': 'https://open.spotify.com/user/sdspot2034'},
 'href': 'https://api.spotify.com/v1/users/sdspot2034',
 'id': 'sdspot2034',
 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b8280153af8b79b3144b24f1291',
   'height': 64,
   'width': 64},
  {'url': 'https://i.scdn.co/image/ab6775700000ee8580153af8b79b3144b24f1291',
   'height': 300,
   'width': 300}],
 'type': 'user',
 'uri': 'spotify:user:sdspot2034',
 'followers': {'href': None, 'total': 9},
 'country': 'IN',
 'product': 'premium',
 'explicit_content': {'filter_enabled': False, 'filter_locked': False},
 'email': 'shreyandas2034@gmail.com'}

In [6]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
# print(yesterday.timestamp()*1e3)
cdc_time = int(yesterday.timestamp()*1e3)

# Get All Songs Played in the last one day

In [7]:
token = spotify_authorisation.get_access_token()
history = spotify.get_listening_history(token, cdc_time)['items']

## EDA over recent history

### Number of songs played

In [8]:
len(history)

18

### Fetch details of a song

#### See available data for a song

In [9]:
history[0].keys()

dict_keys(['track', 'played_at', 'context'])

In [10]:
list(history[0]['track'].keys())

['album',
 'artists',
 'disc_number',
 'duration_ms',
 'explicit',
 'external_ids',
 'external_urls',
 'href',
 'id',
 'is_local',
 'name',
 'popularity',
 'preview_url',
 'track_number',
 'type',
 'uri']

In [11]:
history[0]['track']

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
    'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
    'id': '5pKCCKE2ajJHZ9KAiaK11H',
    'name': 'Rihanna',
    'type': 'artist',
    'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4STHEaNw4mPZ2tzheohgXB'},
    'href': 'https://api.spotify.com/v1/artists/4STHEaNw4mPZ2tzheohgXB',
    'id': '4STHEaNw4mPZ2tzheohgXB',
    'name': 'Paul McCartney',
    'type': 'artist',
    'uri': 'spotify:artist:4STHEaNw4mPZ2tzheohgXB'}],
  'external_urls': {'spotify': 'https://open.s

In [12]:
history[0]['played_at']

'2024-08-01T14:18:07.429Z'

In [13]:
print(history[0]['context'])

{'type': 'playlist', 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1E8DyZXT5yUDnr', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1E8DyZXT5yUDnr'}, 'uri': 'spotify:playlist:37i9dQZF1E8DyZXT5yUDnr'}


# Transform

In [55]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, insert

## Dimensions

### DIM_ARTIST

In [49]:
# stacked list-comprehension
artist_list = [artist for artists in [play['track']['artists'] for play in history] for artist in artists]

In [51]:
df_artists = pd.DataFrame(artist_list)
df_artists = df_artists.drop(columns=['href','uri','external_urls'])
df_artists = df_artists.rename(columns={'id':'spotify_id','name':'artist_name','type':'artist_type'})
df_artists.drop_duplicates(inplace=True)
df_artists

,spotify_id,artist_name,artist_type
0,5pKCCKE2ajJHZ9KAiaK11H,Rihanna,artist
1,5K4W6rqBFWDnAN6FQUkS6x,Kanye West,artist
2,4STHEaNw4mPZ2tzheohgXB,Paul McCartney,artist
3,4gzpq5DPGxSnKTe4SA8HAU,Coldplay,artist
4,04gDigrS5kc9YWfZHwBETP,Maroon 5,artist
5,1Xyo4u8uXC1ZmMpatF05PJ,The Weeknd,artist
6,4IWBUUAFIplrNtaOHcJPRM,James Arthur,artist
8,1buzCmyYZE4kcdLRudsb8V,Mikky Ekko,artist
9,3LtlJprzuq0Ii8p8YFZXai,Peter Sandberg,artist
12,1moxjboGR7GNWYIMWsRjgG,Florence + The Machine,artist


#### Load (Type 1 SCD)

In [75]:
with open('SQL_CREDENTIALS.json', 'r') as file:
    cred = json.loads(file.read())
    
user = cred['user']
pswd = cred['pass']
host = cred['host']

target_engine = create_engine(f'mysql+mysqlconnector://{user}:{pswd}@{host}:3306/spotify_db')

with target_engine.connect() as connection:
    dim_artist = pd.read_sql_table("DIM_ARTIST", con=connection)
    
df_combined = df_artists.join(dim_artist, on=["spotify_id"], how="left", rsuffix='_dim')
df_inserts = df_combined[df_combined['artist_wid'].isnull()][['spotify_id','artist_name','artist_type']]
df_updates = df_combined.dropna(subset=['artist_wid'])

with target_engine.connect() as connection:
    df_inserts.to_sql(name='DIM_ARTIST', con=connection, if_exists='append', index=False)

### DIM_SONG

In [17]:
history[0]['track'].keys()

dict_keys(['album', 'artists', 'disc_number', 'duration_ms', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track_number', 'type', 'uri'])

In [18]:
history[0]['track']

{'album': {'album_type': 'single',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5pKCCKE2ajJHZ9KAiaK11H'},
    'href': 'https://api.spotify.com/v1/artists/5pKCCKE2ajJHZ9KAiaK11H',
    'id': '5pKCCKE2ajJHZ9KAiaK11H',
    'name': 'Rihanna',
    'type': 'artist',
    'uri': 'spotify:artist:5pKCCKE2ajJHZ9KAiaK11H'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4STHEaNw4mPZ2tzheohgXB'},
    'href': 'https://api.spotify.com/v1/artists/4STHEaNw4mPZ2tzheohgXB',
    'id': '4STHEaNw4mPZ2tzheohgXB',
    'name': 'Paul McCartney',
    'type': 'artist',
    'uri': 'spotify:artist:4STHEaNw4mPZ2tzheohgXB'}],
  'external_urls': {'spotify': 'https://open.s